*Load Required Libraries*

In [1]:
# Libraries for data processing
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from pprint import pprint
import tqdm
import datetime

# NLP set-up 
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmaObj = WordNetLemmatizer()

# Libraries for Word Cloud
from wordcloud import WordCloud
from tqdm import tqdm
from collections import Counter

# Libraries for data processing and extraction
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Libraries for visualization
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
%matplotlib inline

# Libraries for Topic Modelling 
import gensim
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tanay\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\tanay\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
C:\Users\tanay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
C:\Users\tanay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gensim\matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices im

In [2]:
df = pd.read_csv('/Users/tanay/Downloads/finalprocessed_data.csv')

In [3]:
df

,Name,Comment,Date,Model,sentiment_score,Source,cleaned_review,sentiment_label
0,Gumpclipper,"Good morning all,I am looking at buying my fir...",2021-10-10,Porsche Macan,5,Pistonheads,good morning looking buying advice essential ...,positive
1,Voodoo Blue,I’m not going to offer an opinion on spec as t...,2021-10-10,Porsche Macan,4,Pistonheads,going offer opinion spec pretty subjective sug...,positive
2,Gumpclipper,Fantastic info. Thank you! Will take a look at...,2021-10-10,Porsche Macan,5,Pistonheads,fantastic info look forum cheer,positive
3,TheBMWDriver,Right now 1% is best you can get on the macan....,2021-10-21,Porsche Macan,4,Pistonheads,right percent best speci find k option make ...,positive
4,neilf,My tip would be to get your name on the deposi...,2021-10-22,Porsche Macan,1,Pistonheads,tip deposit list opc asap way queue decision d...,negative
...,...,...,...,...,...,...,...,...
48940,Binkyjean,The width of the Levante and Grecale are almos...,2022-03-25,Maserati Grecale,2,Forums,width levante inch mirror,negative
48941,Mii,"1,948 mm (76.7 in) is listed on Wiki, should b...",2022-04-02,Maserati Grecale,3,Forums,mm listed wiki width mirror folded http en...,neutral
48942,Harvey Wiggins,"Finally, I found a spec that listed body width...",2022-04-02,Maserati Grecale,3,Forums,finally found spec listed body width mirror wi...,neutral
48943,Mii,"Need to see the gear button control in person,...",2022-04-02,Maserati Grecale,3,Forums,gear button control person non gear lever pref...,neutral


In [9]:
# Filter rows with Model "Porsche Macan"
macan_data = df[df['Model'] == 'Porsche Macan']

macan_data

,Name,Comment,Date,Model,sentiment_score,Source,cleaned_review,sentiment_label
0,Gumpclipper,"Good morning all,I am looking at buying my fir...",2021-10-10,Porsche Macan,5,Pistonheads,good morning looking buying advice essential ...,positive
1,Voodoo Blue,I’m not going to offer an opinion on spec as t...,2021-10-10,Porsche Macan,4,Pistonheads,going offer opinion spec pretty subjective sug...,positive
2,Gumpclipper,Fantastic info. Thank you! Will take a look at...,2021-10-10,Porsche Macan,5,Pistonheads,fantastic info look forum cheer,positive
3,TheBMWDriver,Right now 1% is best you can get on the macan....,2021-10-21,Porsche Macan,4,Pistonheads,right percent best speci find k option make ...,positive
4,neilf,My tip would be to get your name on the deposi...,2021-10-22,Porsche Macan,1,Pistonheads,tip deposit list opc asap way queue decision d...,negative
...,...,...,...,...,...,...,...,...
17099,Wing Commander,I suppose we should be grateful that at least ...,2021-10-02,Porsche Macan,5,Forums,suppose grateful allowing,positive
17100,sd1985,Our first macan was a base on 21s with no Pasm...,2021-10-02,Porsche Macan,3,Forums,base pasm air liked come inferior depends co...,neutral
17101,BanZ,"For gen3, on the uk configurator PASM is not c...",2021-10-02,Porsche Macan,4,Forums,gen uk configurator pasm currently base stand...,positive
17102,Col Lamb,Well that is another reason for avoiding the 2...,2021-10-02,Porsche Macan,1,Forums,reason avoiding litre shout wheel pasm fai...,negative


*LDA Model*

In [12]:
# Create Dictionary
dicton = corpora.Dictionary(macan_data['cleaned_review'].str.split())

# Create Corpus
texts = macan_data['cleaned_review'].str.split()

# Term Document Frequency
corpus = [dicton.doc2bow(text) for text in texts]

print(corpus[:1][0][:30])

[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)]


In [13]:
# Build a LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=dicton,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [14]:
# Print the keywords in topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.057*"gen" + 0.040*"gts" + 0.023*"turbo" + 0.015*"engine" + 0.010*"v" + '
  '0.009*"gap" + 0.009*"look" + 0.008*"difference" + 0.008*"bonnet" + '
  '0.008*"l"'),
 (1,
  '0.033*"mpg" + 0.022*"mile" + 0.016*"fuel" + 0.011*"coating" + '
  '0.010*"engine" + 0.010*"trip" + 0.009*"petrol" + 0.009*"audi" + '
  '0.008*"ceramic" + 0.008*"average"'),
 (2,
  '0.039*"look" + 0.024*"like" + 0.024*"black" + 0.024*"colour" + '
  '0.014*"great" + 0.013*"wheel" + 0.012*"good" + 0.011*"spec" + 0.010*"red" + '
  '0.009*"thanks"'),
 (3,
  '0.021*"sport" + 0.015*"seat" + 0.014*"way" + 0.013*"gts" + '
  '0.011*"suspension" + 0.011*"standard" + 0.010*"air" + 0.010*"wheel" + '
  '0.010*"ride" + 0.009*"like"'),
 (4,
  '0.023*"http" + 0.020*"uk" + 0.020*"www" + 0.018*"com" + 0.012*"co" + '
  '0.012*"good" + 0.009*"review" + 0.008*"n" + 0.008*"thanks" + 0.007*"guy"'),
 (5,
  '0.030*"opc" + 0.022*"service" + 0.019*"k" + 0.017*"warranty" + '
  '0.014*"month" + 0.013*"year" + 0.011*"cost" + 0.011*"oil" + 0

In [15]:
# Calculate the overall Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dicton, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.5839890390674424


In [16]:
# Supporting Model 
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dicton, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [18]:
grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 12
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dicton, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results)
    model_results
    pbar.close()

100%|██████████| 600/600 [3:54:13<00:00, 23.42s/it]  


In [19]:
macan_results= pd.DataFrame(model_results)
macan_results

,Validation_Set,Topics,Alpha,Beta,Coherence
0,75% Corpus,2,0.01,0.01,0.649218
1,75% Corpus,2,0.01,0.31,0.659738
2,75% Corpus,2,0.01,0.61,0.659738
3,75% Corpus,2,0.01,0.91,0.659738
4,75% Corpus,2,0.01,symmetric,0.659738
...,...,...,...,...,...
595,100% Corpus,11,asymmetric,0.01,0.574359
596,100% Corpus,11,asymmetric,0.31,0.512749
597,100% Corpus,11,asymmetric,0.61,0.528383
598,100% Corpus,11,asymmetric,0.91,0.524443


In [20]:
macan_results.to_csv('/Users/tanay/Downloads/macan_cohenrencef.csv', index=False, encoding="utf-8-sig")

In [21]:
# Number of Topics 3 has the highest coherence score (0.67), therefore we continue with that
num_topics = 3

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dicton,
                                           num_topics=num_topics, 
                                           random_state=42,
                                           chunksize=100,
                                           passes=15,
                                           alpha=0.02,
                                           eta=0.9)

In [22]:
# Print the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.010*"look" + 0.009*"wheel" + 0.008*"like" + 0.007*"sport" + 0.006*"black" '
  '+ 0.006*"colour" + 0.006*"good" + 0.006*"seat" + 0.005*"gen" + '
  '0.005*"light"'),
 (1,
  '0.013*"opc" + 0.009*"service" + 0.006*"warranty" + 0.006*"month" + '
  '0.006*"year" + 0.005*"k" + 0.005*"price" + 0.005*"dealer" + 0.005*"day" + '
  '0.004*"good"'),
 (2,
  '0.010*"gts" + 0.008*"mile" + 0.008*"k" + 0.007*"like" + 0.006*"mpg" + '
  '0.006*"engine" + 0.005*"turbo" + 0.005*"gen" + 0.005*"road" + 0.004*"year"')]


In [23]:
# Visualize the topics
pyLDAvis.enable_notebook()
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, dicton)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.101188 -0.062593       1        1  36.754954
0     -0.114657 -0.042853       2        1  34.084569
2      0.013470  0.105446       3        1  29.160478, topic_info=          Term         Freq        Total Category  logprob  loglift
115        opc  1363.000000  1363.000000  Default  30.0000  30.0000
199    service   909.000000   909.000000  Default  29.0000  29.0000
2495     black   617.000000   617.000000  Default  28.0000  28.0000
215        mpg   529.000000   529.000000  Default  27.0000  27.0000
817     colour   605.000000   605.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
16    probably   218.856452   495.007192   Topic3  -5.9444   0.4162
43       going   237.440659   648.749729   Topic3  -5.8629   0.2272
534        day   263.339828   919.661773   Topic3  -5.7593  -0.0182
371      price   232.399166   774.271091   Topic3  -5.8843   0.0289
83        look   240.546009  1394.993254   Topic3  -5.8499  -0.5254

[223 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3950      1  0.005863      acc
3950      2  0.926393      acc
3950      3  0.064496      acc
5289      1  0.983811  admiral
5289      2  0.010466  admiral
...     ...       ...      ...
78        2  0.003649      www
78        3  0.003649      www
181       1  0.560750     year
181       2  0.088540     year
181       3  0.350350     year

[531 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])